In [1]:
import json
import os
import re
import pandas as pd

In [2]:
# Load data
folder_path = "../results/base/"
data = []

for root, dirs, files in os.walk(folder_path):
    for filename in files:
        if filename.endswith(".json"):
            file_path = os.path.join(root, filename)
            try:
                with open(file_path, "r") as f:
                    entry = json.load(f)
                    data.append(entry)
            except json.JSONDecodeError as e:
                print(f"Skipped invalid JSON: {file_path} ({e})")

print(f"Loaded {len(data)} valid JSON result files from '{folder_path}' and its subfolders.")

Loaded 0 valid JSON result files from 'results/base/' and its subfolders.


In [5]:
# Feature extraction helpers
def avg_sentence_length(text):
    sentences = re.split(r'[.!?]', text)
    sentences = [s for s in sentences if s.strip()]
    words = sum(len(s.split()) for s in sentences)
    return words / len(sentences) if sentences else 0

def count_bullets(text):
    return len(re.findall(r"[-*]\s", text))

def count_headings(text):
    return len(re.findall(r"^#+\s", text, flags=re.MULTILINE))

# Process the dialogue data
records = []

for entry in data:
    dialogue = entry.get("dialogue", [])
    for turn in dialogue:
        role = turn["role"]
        agent_id = int(role.split("_")[1])
        content = turn["content"]
        records.append({
            "agent_id": agent_id,
            "agent_model": entry.get("model_general_name_0") if agent_id == 0 else entry.get("model_general_name_1"),
            "sentence_length": avg_sentence_length(content),
            "content": content,
            "bullet_count": count_bullets(content),
            "heading_count": count_headings(content),
        })

# Create DataFrame and compute summary
df = pd.DataFrame(records)
grouped = df.groupby("agent_model")
summary_mean = grouped.mean(numeric_only=True).reset_index()
summary_std = grouped.std(numeric_only=True).reset_index()
summary_df = summary_mean.merge(summary_std, on="agent_model", suffixes=("_mean", "_std"))

# Combine mean and std columns into formatted strings
for feature in ['sentence_length', 'bullet_count', 'heading_count']:
    summary_df[f"{feature} (mean ± std)"] = summary_df[f"{feature}_mean"].round(1).astype(str) + " ± " + summary_df[f"{feature}_std"].round(1).astype(str)

# Final DataFrame with combined results
final_df = summary_df[["agent_model"] + [f"{f} (mean ± std)" for f in ['sentence_length', 'bullet_count', 'heading_count']]]
final_df.columns = ["Model", "Avg Sentence Length", "Bullet Count", "Heading Count"]

print(final_df.to_latex(index=False))

\begin{tabular}{llll}
\toprule
Model & Avg Sentence Length & Bullet Count & Heading Count \\
\midrule
deepseek & 14.7 ± 8.8 & 23.5 ± 20.9 & 2.2 ± 3.8 \\
llama & 17.4 ± 3.8 & 4.9 ± 9.5 & 0.7 ± 2.1 \\
qwen & 9.1 ± 6.5 & 33.8 ± 27.8 & 4.8 ± 4.5 \\
\bottomrule
\end{tabular}

